In [1]:
import numpy as np
import pandas as pd
import glob as gb 
import os

### Состояние склада на каждый день

Создание имени файла

In [2]:
def create_filenames(letter, number):
    
    c = chr(ord('0') + number)
    str_i = 'MS-' + letter + c + '-inventory.csv'
    str_sl = 'MS-' + letter + c + '-sell.csv'
    str_spl = 'MS-' + letter + c + '-supply.csv'
    str_d = 'MS-' + letter + c + '-daily.csv'
    str_st = 'MS-' + letter + c + '-stolen'
    return str_i, str_sl, str_spl, str_d, str_st

Количество магазинов в штате

In [3]:
def num_shop(state):
    
    if state == 'b' or state == 'm':
        max = 2
    elif state == 's':
        max = 5
    return max

Чтение файлов 

In [20]:
def get_files(state, directory):
    max = num_shop(state)
    files = {}
    path = {}
    shop = 0
    while shop < max:
        c = chr(ord('0') + shop + 1)
        path[shop] = gb.glob(directory + state + c + '*.csv')
        print(path[shop])
        files[shop] = {}
        for j in range(0, len(path[shop])):
            print("opening: ", path[shop][j])
            files[shop][j]= pd.read_csv(path[shop][j], index_col='date', parse_dates=True, dayfirst=True)
        shop = shop + 1
    return files, max

Подсчет общего количества проданного товара за день

In [21]:
def sell_upg(sell):
    
    upg_tmp = pd.DataFrame(columns=['sku_num'], index = sell.index)
    upg_tmp['sku_num'] = sell['sku_num'].apply(lambda x: 0 if x.find('ap') == 6 else 1)
    daily = pd.crosstab(upg_tmp.index, upg_tmp['sku_num'], margins = True)
    daily = daily.drop(['All'])
    daily_new = pd.DataFrame(columns=['apple', 'pen'], index = upg_tmp.index.drop_duplicates())
    daily_new['apple'] = daily[0].values
    daily_new['pen'] = daily[1].values
        
    return daily_new

In [6]:
INV = 0
SELL = 1
SPL = 2
DAILY = 3
STOLEN = 4

### Task 1

In [46]:
def daily_store(sell, supply):

    daily_new = pd.DataFrame(columns=['apple', 'pen'], index = sell.index)
    daily_new['apple'] = -sell['apple']
    daily_new['pen'] = -sell['pen']

    df = pd.concat([supply, daily_new]).sort_index()
    df = df.resample('D').sum()

    l = df.index.size
    df['apple_sum'] = df['apple'].rolling(str(l) + 'D').sum()
    df['pen_sum'] = df['pen'].rolling(str(l) + 'D').sum()
    df = df.drop(['apple', 'pen'], axis=1)
    df = df.rename(columns={'apple_sum': 'apple', 'pen_sum': 'pen'})
    print("daily_store\n", df.head())
      
    return df
    

### Task 2

In [24]:
def stolen(sell, supply, inventory):
    
    st = pd.DataFrame(columns=['apple', 'pen'], index = inventory.index)
    sell = sell.resample('M').sum()
    supply = supply.resample('M').sum()
    inv = pd.DataFrame(columns=['apple', 'pen'], index = inventory.index)
    inv['apple'] = inventory['apple'] - inventory['apple'] .shift(1)
    inv.loc[inv.index[0], 'apple'] = inventory.loc[inventory.index[0], 'apple']
    inv['pen'] = inventory['pen'] - inventory['pen'] .shift(1)
    inv.loc[inv.index[0], 'pen'] = inventory.loc[inventory.index[0], 'pen']
    st['apple'] = inv['apple'] + sell['apple'] - supply['apple']
    st['pen'] = inv['pen'] + sell['pen'] - supply['pen'] 
    return st.abs()

In [47]:
for i in {'b', 'm', 's'}:
#for i in {'b'}:              # для проверки
    files, max = get_files(i, './input/*')
    shop = 0
    while shop < max:
        files[shop][DAILY] = sell_upg(files[shop][SELL])
        
        dly = daily_store(files[shop][DAILY], files[shop][SPL])
        name = create_filenames(i, shop + 1)[DAILY]
        dly.to_csv('./answer/' + name, index=False)
        print("daily\n", dly.head(15))
        stl = stolen(files[shop][DAILY], files[shop][SPL], files[shop][INV]) 
        name = create_filenames(i, shop + 1)[STOLEN]
        stl.to_csv('./answer/' + name, index=False)
        print("stolen\n", stl.head(15))
        shop = shop + 1

['./input\\MS-b1-inventory.csv', './input\\MS-b1-sell.csv', './input\\MS-b1-supply.csv']
opening:  ./input\MS-b1-inventory.csv
opening:  ./input\MS-b1-sell.csv
opening:  ./input\MS-b1-supply.csv
['./input\\MS-b2-inventory.csv', './input\\MS-b2-sell.csv', './input\\MS-b2-supply.csv']
opening:  ./input\MS-b2-inventory.csv
opening:  ./input\MS-b2-sell.csv
opening:  ./input\MS-b2-supply.csv
daily_store
               apple     pen
date                       
2006-01-01  33271.0  2574.0
2006-01-02  31409.0  2431.0
2006-01-03  29529.0  2260.0
2006-01-04  27732.0  2107.0
2006-01-05  25790.0  1974.0
daily
               apple     pen
date                       
2006-01-01  33271.0  2574.0
2006-01-02  31409.0  2431.0
2006-01-03  29529.0  2260.0
2006-01-04  27732.0  2107.0
2006-01-05  25790.0  1974.0
2006-01-06  23892.0  1825.0
2006-01-07  21997.0  1687.0
2006-01-08  20161.0  1567.0
2006-01-09  18293.0  1442.0
2006-01-10  16414.0  1286.0
2006-01-11  14618.0  1135.0
2006-01-12  12703.0   995.0
20

### Task 3

In [36]:
def year_table():
    states = ['b', 'm', 's']
    nshops = [2, 2, 5]
    res = {}
    t = {}
    sold = {}
    for state in range (0, 3):
        for shop in range (0, nshops[state]):
            sold += sell_upg(files[shop][SELL]).resample('Y').sum()
            stolen += stolen(files[shop][SELL], files[shop][SPL], files[shop][INV]).resample('Y').sum()
        t[i] = sold
        t[i].insert(3, 'apple_stolen', stolen[1])
        t[i].insert(5, 'pen_stolen', stolen[2])
        t[i].insert(1, 'state', states[state])
        #я хз как это написать не криво, завтра поищу другие вараинты
        #for j in range(0, nshops[state]):
            #t[i]['state'][j] = states[state]
        
        
    res = pd.concat([t[0], t[1], t[2]]).sort_index()
    print(res)
    return res

In [37]:
year_table().head()

ValueError: Invalid frequency: Y

In [16]:
#def check_res():
states = ['b', 'm', 's']
for state in states:
    right_ansm, maxou = get_files(state, './output/*')
    our_ans, maxa = get_files(state, './answer/*') 
    for i in range (0, maxou):
        if (right_ansm[i] != our_ans[i]):
            print('ответы не сошлись')

['./output\\MS-b1-daily.csv', './output\\MS-b1-steal.csv']
['./output\\MS-b2-daily.csv', './output\\MS-b2-steal.csv']
['./answer\\MS-b1-daily.csv']


ValueError: 'date' is not in list